In [2]:
import wfdb
import pywt
import seaborn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.python.keras import Model, Input
from tensorflow.python.keras.layers import LSTM, Dropout, Dense,Attention, multiply
from tensorflow.python.keras.models import Sequential
#from tensorflow.python.keras.optimizers import SGD,Adam
from tensorflow.python.keras.layers.core import *
#from tensorflow.python.keras.utils.vis_utlis import plot_model
from wfdb import processing
import pandas as pd
from sklearn.model_selection import train_test_split

#record_ecg= wfdb.rdrecord('C:/Users/JoyceYing/siteEnv/machineTest/physionet.org/files/picsdb/1.0.0/infant{}_ecg'.format(i))
#record_resp= wfdb.rdrecord('C:/Users/JoyceYing/siteEnv/machineTest/physionet.org/files/picsdb/1.0.0/infant{}_resp'.format(i))
#r_peaks= wfdb.rdann('C:/Users/JoyceYing/siteEnv/machineTest/physionet.org/files/picsdb/1.0.0/infant{}_ecg'.format(i), 'qrsc')
#resp_peaks= wfdb.rdann('C:/Users/JoyceYing/siteEnv/machineTest/physionet.org/files/picsdb/1.0.0/infant{}_resp'.format(i), 'resp')
#List of record names to be used for analysis
#record_names = ['C:/Users/JoyceYing/siteEnv/machineTest/physionet.org/files/picsdb/1.0.0/infant1', 'C:/Users/JoyceYing/siteEnv/machineTest/physionet.org/files/picsdb/1.0.0/infant2']

# Loop through each record name
#for record_name in record_names:
   # record_ecg= wfdb.rdrecord(record_name+'_ecg'.format(1))
   # record_resp= wfdb.rdrecord(record_name+'_resp'.format(1))
   # r_peaks= wfdb.rdann(record_name+'_ecg'.format(1), 'qrsc')
    #resp_peaks= wfdb.rdann(record_name+'_resp'.format(1), 'resp')
    
for i in range(1,11):
    heart_data = []
    resp_data = []
    
    record_ecg= wfdb.rdrecord('./infant{}_ecg'.format(i))
    record_resp=wfdb.rdrecord('./infant{}_resp'.format(i))
    ecg_signal = record_ecg.p_signal
    
    r_peaks= wfdb.rdann('./infant{}_ecg'.format(i),'qrsc')
    resp_peaks=wfdb.rdann('./infant{}_resp'.format(i),'resp')
    

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Ding/Documents/tset01/infant1_resp.hea'

In [ ]:
def compute_hr(sig_len, qrs_ins, fs):
    heart_rate=np.full(sig_len,np.nan,dtype="float32")

    if len(qrs_inds)<2:
        return heart_rate

    for i in range(0, len(qrs_inds)-2):
            a=qrs_inds[i]
            b=qrs_inds[i+1]
            c=qrs_inds[i+2]
            rr=(b-a)*(1.0/fs)*1000
            hr=60000.0/rr
            heart_rate[b+1:c+1]=hr

            heart_rate[qrs_inds[-1]:]=heart_rate[qrs_inds[-1]]

            return heart_rate

In [ ]:
def compute_hr_custom(r_peak_1,r_peak_2,fs):
     a=r_peak_1
     b=r_peak_2
     rr=(b-a)*(1.0/fs)
     hr=60/rr

     return hr

In [ ]:
heart_rate = processing.compute_hr(len(record_ecg.p_signal),r_peaks.sample,record_ecg.fs)
breathing_rate = processing.compute_hr(len(record_resp.p_signal),resp_peaks.sample,record_resp.fs)

In [5]:
custom_hr=compute_hr_custom(r_peaks.sample[0],r_peaks.sample[1],record_ecg.fs)
wfdb_hr =next(x for x in heart_rate if not np.isnan(x))

print('own custom function'+str(round(custom_hr,2)))
print('wfdb function'+str(round(wfdb_hr,2)))

own custom function159.57
wfdb function159.57


In [6]:
ecg_samp_interval=1/record_ecg.fs
resp_samp_interval=1/record_resp.fs

print("ECG slp frequecy:",record_ecg.fs, "Hz")
print("ECG slp interval every:",ecg_samp_interval, "sec")
print("RESP slp frequecy:",ecg_samp_interval, "sec")
print("RESP slp interval every:",ecg_samp_interval, "sec")

ECG slp frequecy: 500 Hz
ECG slp interval every: 0.002 sec
RESP slp frequecy: 0.002 sec
RESP slp interval every: 0.002 sec


In [7]:
time_ecg=np.arange(record_ecg.p_signal.shape[0])*ecg_samp_interval
time_resp=np.arange(record_resp.p_signal.shape[0])*resp_samp_interval

In [23]:

    heart_date={
        'Infant:': i,
        'Heart_rate': heart_rate,
        'Time(sec)': time_ecg,

    }
    resp_data={
        'Infant:': i,
        'Heart_rate': breathing_rate,
        'Time(sec)': time_resp, 
    }
    heart_df = pd.DataFrame.from_dict(heart_date)
    resp_df = pd.DataFrame.from_dict(resp_data)

In [9]:
print('Respiratory DF')
heart_df
resp_df

Respiratory DF


,Infant:,Heart_rate,Time(sec)
0,10,NaN,0.00
1,10,NaN,0.02
2,10,NaN,0.04
3,10,NaN,0.06
4,10,NaN,0.08
...,...,...,...
8508149,10,93.75,170162.98
8508150,10,93.75,170163.00
8508151,10,93.75,170163.02
8508152,10,93.75,170163.04


In [22]:
for i in range(1,11):
    record = wfdb.rdrecord('C:/Users/JoyceYing/siteEnv/machineTest/physionet.org/files/picsdb/1.0.0/infant{}_ecg'.format(i))
    ecg_record = wfdb.rdann('C:/Users/JoyceYing/siteEnv/machineTest/physionet.org/files/picsdb/1.0.0/infant{}_ecg'.format(i),'atr')
    annotation = wfdb.rdann('C:/Users/JoyceYing/siteEnv/machineTest/physionet.org/files/picsdb/1.0.0/infant{}_ecg'.format(i),'qrsc')
    sample = ecg_record.sample
    print(sample)
    
    X = record.p_signal
    y = annotation.sample

    
    if len(X) < 2 or len(y) < 2:

    
        # Split the data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, r_peaks, test_size=0.2, random_state=42)

        # Save the data as .npy files
        np.save('X_train.npy', X_train)
        np.save('y_train.npy', y_train)
        np.save('X_test.npy', X_test)
        np.save('y_test.npy', y_test)


[  492843   864128   984499  1208687  1924068  2061665  3277415  3671197
  4092936  4279525  4474989  4960095  6131606  6380121  7580334  8839650
  9948807 11393804 12506763 13091675 13870835 14742871 15166275 15326326
 15483730 15973421 16224718 17529945 18150760 18243747 19242692 19470997
 19937850 20253096 20736260 21234763 21333504 21676097 21979795 22498931
 22737631 23086767 23304355 23903101 24344758 24437056 24768375 25644652
 25811592 26068648 26316024 27050032 27513120 27796679 28875005 29505771
 29862851 30226051 31105267 31306300 31430173 31692956 32108501 32792265
 33124169 34681283 35165114 35667578 36541041 36957758 37056744 37365509
 37940274 38815618 39722902 40109190 40207931]
[ 1117892  2105380  3295899  4990848  5301183  6356582  9912207 12390208
 12685106 15407103 15691703 16107411 16312219 16563763 17117123 18368408
 20442789 21425027 21867676 23915651 24619121 24924093 26331412 26866660
 27575395 28658345 28978755 29429370 30510130 30786083 31302141 31820539
 347